# 循环神经网络

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from tensorflow.keras import layers, Model, Input

## 序列
具有先后顺序的数据一般叫作序列(Sequence). 我们把文字编码为数值的过程叫作**Word Embedding**.

one-hot编码的优缺点:
- 简单直观，编码过程不需要学习和训练;
- 但高维度而且极其稀疏的，大量的位置为0，计算效率较低, 忽略了单词先天具有的语义相关性;

余弦相关度(Cosine similarity), 衡量词向量(word vector)之间相关度:
$$similarity(a, b) \triangleq \frac {a \cdot b}{|a|\cdot|b|}$$

### Embedding层
单词的表示层叫作Embedding层, 负责把单词编码为某个词向量𝒗

$$v = f_{\theta}(i|N_{vocab}, n)$$
单词数量记为$N_{vocab}$, $v的长度为n$, $i$表示单词编号, 如2 表示“I”，3 表示“me”等.

In [ ]:
x = tf.range(10)  # 代表10个不同单词的编码

x = tf.random.shuffle(x)
# 10个单词, 每个单词用长度4 的向量表示
net = layers.Embedding(10, 4)
out = net(x)
out

In [ ]:
net.get_weights()

### 预训练的词向量

应用的比较广泛的预训练模型:Word2Vec 和GloVe模型.利用已预训练好的模型参数初始化Embedding层.

In [ ]:
def load_embed(path):
    # 建立映射关系: 单词: 词向量(长度50))
    embedding_map = {}
    with open(path, encoding='utf8') as f:
        for line in f.readlines():
            l = line.split()
            word = l[0]
            coefs = np.asarray(l[1:], dtype='float32')
            embedding_map[word] = coefs
    return embedding_map

In [ ]:
embedding_map = load_embed('glove.6B.50d.txt')
print('Found %s word vectors.' % len(embedding_map))

In [ ]:
embedding_map['the']

### 20newsgroups 测试

In [ ]:
from sklearn import datasets
# 加载20newsgroups数据集
news20 = datasets.fetch_20newsgroups()

In [ ]:
news20.keys()

In [ ]:
category = news20.target_names  # 一共20类不同的新闻
category

In [ ]:
labels = news20['target']  # 每条新闻分属的类别

In [ ]:
len(news20['data'])

In [ ]:
news20['data'][0], category[news20['target'][0]]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
MAX_NUM_WORDS = 20000  # 最多保留 20000-1 个不同的单词
MAX_SEQUENCE_LENGTH = 1000  # 每个序列长度
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 50  # 用50维向量表示一个单词

In [ ]:
Tokenizer?

In [ ]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)  #  Only the most common `num_words-1` words will be kept.

In [ ]:
# Updates internal vocabulary based on a list of texts
tokenizer.fit_on_texts(news20['data'])
sequences = tokenizer.texts_to_sequences(news20['data'])  # 语句 -> 单词序列号组成的sequences

In [ ]:
# matrix = tokenizer.texts_to_matrix(news20['data'])
# matrix.shape  # (11314, 20000)  稀疏矩阵

In [ ]:
sequences[0]

In [ ]:
# 将sequences 转成文本list
# tokenizer.sequences_to_texts(sequences)

In [ ]:
# 将单词映射为 index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
word_index_list = list(word_index.items())

In [ ]:
# 从1开始编码 用0代表填充
word_index_list[:10]  # news20group 出现频率最高的10个单词

In [ ]:
word_index_list[19998]

In [ ]:
# Pads sequences to the same length.
pad_sequences?

In [ ]:
# 每条新闻都被编码成 等长的 用数字表示的 序列
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
data.shape

In [ ]:
np.max(data), np.min(data) # 

In [ ]:
from sklearn.model_selection import train_test_split

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=VALIDATION_SPLIT, random_state=0) 

In [ ]:
X_train.shape, y_test.shape

In [ ]:
# 将 单词序号-> 单词向量(长度50)
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    # 根据glove.6B.50d 将单词转为词向量
    embedding_vector = embedding_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# new20group中最常用的19999 词向量 + 填充 + unknow
embedding_matrix.shape

In [ ]:
embedding_matrix[-1]

In [ ]:
layers.Embedding?

In [ ]:
embedding_layer = layers.Embedding(
    num_words, EMBEDDING_DIM,
    weights = [embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [ ]:
sequence_input = Input((MAX_SEQUENCE_LENGTH, ), dtype=tf.int32)
embedded_sequences = embedding_layer(sequence_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
preds = layers.Dense(len(category), activation='softmax')(x)

model = Model(inputs=sequence_input, outputs=preds)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, batch_size=128, epochs=15, validation_data=(X_test, y_test))

In [ ]:
plt.plot(np.linspace(1, 15, 15), hist.history['loss'], label='loss')
plt.plot(np.linspace(1, 15, 15), hist.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(np.linspace(1, 15, 15), hist.history['accuracy'], label='accuracy')
plt.plot(np.linspace(1, 15, 15), hist.history['val_accuracy'], label='val_accuracy')
plt.legend()

## 循环神经网络


$$h_t = \sigma(W_{xh}x_t + W_{hh}h_{t-1} + b)$$
在每个时间戳$t$, 网络层接受当前时间戳的输入$x_t$和上一个时间戳的网络状态向量$h_{t-1}$,经过
$$h_t = f_{\theta}(h_{t-1}, x_t)$$
变换后得到当前时间戳的新状态向量$h_t$. 在每个时间戳上, 网络层均有输出$o_t = g_{\phi}(h_t)$

对于这种网络结构，我们把它叫做循环网络结构(Recurrent Neural Network，简称RNN)。